In [1]:
import pandas as pd

In [2]:
file_path = '~/code/TechLah/RevuSum/data/processed_kswdf.csv'
hotelreviews = pd.read_csv(file_path)
df = hotelreviews.copy()

In [5]:
df = df.groupby('Label', group_keys=False).apply(lambda x: x.sample(10000))

df['Label'].value_counts()

Label
0    10000
1    10000
Name: count, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
X = df['Review']
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X,y, shuffle=True,test_size=0.3)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

vectorizer = CountVectorizer()

# Fit and transform the 'clean_text' column
features = vectorizer.fit_transform(df.Review)

# Get the feature names
feature_names = vectorizer.get_feature_names_out()

# Create a DataFrame with the features as column names
X_bow = pd.DataFrame(features.toarray(), columns=feature_names)

In [8]:
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

# Feature/Target
X = X_bow
y = df['Label']

# Pipeline vectorizer + Naive Bayes
pipeline_naive_bayes = make_pipeline(
    CountVectorizer(),
    MultinomialNB()
)

# Define the grid of parameters
parameters = {
    'countvectorizer__ngram_range': ((1,2), (3,4)),
    'multinomialnb__alpha': (0.1,1)
}

# Perform Grid Search
grid_search = GridSearchCV(
    pipeline_naive_bayes,
    parameters,
    scoring = "recall",
    cv = 5,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(df['Review'],df['Label'])

# Best score
print(f"Best Score = {grid_search.best_score_}")

# Best params
print(f"Best params = {grid_search.best_params_}")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best Score = 0.9394
Best params = {'countvectorizer__ngram_range': (1, 2), 'multinomialnb__alpha': 1}


In [9]:
cv_nb = cross_validate(
    pipeline_naive_bayes,
    df['Review'],
    df['Label'],
    scoring = "accuracy",
    cv=10
)

round(cv_nb['test_score'].mean(),2)

0.93

In [10]:
Good_test = "Great room and the view was amazing. The buffet breakfast had so many options and so much of everything as well. \
    The shops under the hotel were really good and also lots of little eating places too.\
        Raffles is a few mins walk and you can walk to Clarke Quay as well.\
            Perfect location and would highly recommend."
            
Bad_test = "the view was horrible. breakfast had so little options and so little of everything. \
    The shops under the hotel were really terrible and very little eating places too."

In [11]:
from sklearn.metrics import accuracy_score

pipeline_naive_bayes = make_pipeline(
    CountVectorizer(ngram_range=(1,2)),
    MultinomialNB(alpha=1)
)

nb_model = pipeline_naive_bayes.fit(X_train,y_train)

y_pred = nb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9336666666666666


In [12]:
predictions = nb_model.predict_proba([Good_test])
prediction_label = pipeline_naive_bayes.predict([Good_test])
predictions,prediction_label

(array([[9.14392822e-22, 1.00000000e+00]]), array([1]))